#### Общие правила
- Сделанная домашняя работа - jupyter-тетрадка.
- Пожалуйста, присваивайте блокам кода номера и/или заголовки, соответствующие заданиям. Код должен содержать комментарии, ход мысли и решения должен быть понятен из вашей тетрадки
- Максимальный балл за домашнюю работу - 20. Вы не обязаны выполнять все пункты, чтобы набрать максимальное количество баллов
- Можно использовать чужой код, но обязательно ставьте ссылки на источник
- **Deadline** - 25.04.2019

Бакшандаева Дарья  
Подрядчикова Мария  
Лисицына Анастасия

## AGRR: Automatic Gapping Resolution for Russian

#### Краткий ликбез
Примеры гэппинга в русском
 - _Ей он рассказывает одно, а нам — совершенно другое_  
 - _Кто любит арбуз, а кто — свиной хрящик_  
 - _Дайте мне две пятерки, а я вам десятку_
 - _Тогда я принял ее за итальянку, а его за шведа_
 

train + dev включают в себя больше 20 тысяч примеров, примерно треть из них иллюстрирует явление гэппинга в русском.  <br>
Гэппинг - явление, при котором в одной из сочиненных (иногда подчиненных) клауз, опускается повторяющийся предикат, но остаются выраженными его участники.
 - _Тогда я принял ее за итальянку, а его ~~принял~~ за шведа_


Терминология для описания элементов гэппинга разнится, мы используем следующую:

При разметке предложения мы выделяем следующие элементы (терминология разнится в разных источниках, бояться не нужно):
- ремнанты **R1** и **R2** - участники эллиптированного предиката (в нашем примере _его_ и _за шведа_)
- корреляты ремнантов  **cR1** и **cR2** - соответсвующие группы в полной клаузе (в нашем примере _ее_ и _за итальянку_)
- позиция опущенного (эллиптированного) предиката  **V**
- вершина конролирующего предиката **cV** (в нашем примере _принял_ из первой клаузы)

Посмотрите на пример со скобочной разметкой:

 - _Тогда я  **cV[** принял **cV]**  **cR1[** ее **cR1]**  **cR2[** за итальянку **cR2]**, а  **R1[** его **R1]**   **V[]**  **cR2[** за шведа **cR2]**._

Есть много теоретических работ, посвященных этому виду эллипсиса (прикладных решений практически нет). Желающим: 

- Лингвистическая сторона дела:
[Ross 1970](https://babel.ucsc.edu/~hank/Ross_Gapping.pdf) 
[Jackendoff 1971](https://babel.ucsc.edu/~hank/jackendoff_related_rules.pdf) 
[Steedman 1989](https://repository.upenn.edu/cgi/viewcontent.cgi?article=1861&context=cis_reports) 
[Coppock 2001](https://www.linguistics.northwestern.edu/documents/award-winners/linguistics-undergraduate-award-past-winner-coppock.pdf)
[Merchant 2016](http://home.uchicago.edu/merchant/pubs/ellipsis.revised.pdf) 

- Подходы к решениям:
[Bogdanov  2012](http://www.dialog-21.ru/media/1316/89.pdf) 
[Droganova 2018](http://www.aclweb.org/anthology/W18-6006) 
[Schuster 2018](https://arxiv.org/pdf/1804.06922.pdf) 



**NB**: Данные собраны, размечены и подготовлены к обучению, так что если вас пугает, что вы не очень понимаете явление, с которым придется работать, это не так критично, не пугайтесь. <br>
Задачу можно воспринимать как классификацию последовательности, в этом смысле она не отличается от задачи NER или POS-tagging. 

 
### Формат данных
Скобочная запись удобна для оценки разметки глазами. Сама разметка хранится в следующем виде:
csv файл, колонки разделены табуляцией.
Первая колонка - сам текст. Вторая колонка - 0 или 1 (отсутсвие или наличие гэппинга в предложении).
Следующие 6 колонок содержат оффсеты подстрок, соответсвующих элементам гэппинга(**cV, cR1, cR2, V, R1, R2**). Ячейка может быть пустой (не все элементы обязательны), может содержать как одну пару оффсетов (разделены через двоеточие), так и несколько пар (разделены через пробел, порядок пар не важен).
Посмотрите на пример:
 
**Input**  
>Аналогичным образом, среднегодовой прирост ВВП на душу населения, который в странах, расположенных к югу от Сахары, составлял в период с 1965 по 1973 год 3 процента, упал с 1980 до 1986 года на 2,8 процента, в 1987 году - на 4,4 процента и в 1989 году - на 0,5 процента.

**Output**  

| class | cV  | cR1  | cR2 | V | R1  | R2  |
| :---  | :---  | :---  | :---  | :---  | :---  | :---  |
| 1| 166:170 | 171:190 | 191:206  | 222:222 254:254  | 208:219 240:251 | 222:237 254:269 |


Такая разметка соответствует следующей скобочной записи:

 - _Аналогичным образом, среднегодовой прирост ВВП на душу населения, который в странах, расположенных к югу от Сахары, составлял в период с 1965 по 1973 год 3 процента,  **cV[** упал **cV]**  **cR1[** с 1980 до 1986 года **cR1]**  **cR2[** на 2,8 процента **cR2]**,   **R1[** в 1987 году **R1]** — **V[]** **R2[** на 4,4 процента **R2]** и  **R1[** в 1989 году **R1]** —  **V[]** **R2[** на 0,5 процента **R2]**._

### 1. Подготовка данных (2 балла)
Скачайте данные (train, dev, test). Для работы с таблицей используйте pandas. 
Используйте параметр quoting - иначе pandas вольно обращается с кавычками в тексте и оффсеты могут съехать
> import pandas as pd <br>
import csv <br>
> train = pd.read_csv('train.csv',sep = '\t', quoting=csv.QUOTE_NONE) 

Посчитайте, сколько в каждом из файлов примеров из класса 0 и из класса 1.


Скачиваем.

In [1]:
import wget
import pandas as pd 
import csv 

In [6]:
wget.download("https://raw.githubusercontent.com/artemovae/ML-for-compling/master/homeworks/HW2/test.csv")
wget.download("https://raw.githubusercontent.com/artemovae/ML-for-compling/master/homeworks/HW2/train.csv")
wget.download("https://raw.githubusercontent.com/artemovae/ML-for-compling/master/homeworks/HW2/dev.csv")

'dev.csv'

Открываем в пандас.

In [2]:
train = pd.read_csv('train.csv',sep = '\t', quoting=csv.QUOTE_NONE)
test = pd.read_csv('test.csv',sep = '\t', quoting=csv.QUOTE_NONE)
dev = pd.read_csv('dev.csv',sep = '\t', quoting=csv.QUOTE_NONE)

In [3]:
train.head()

,text,class,cV,cR1,cR2,V,R1,R2
0,"Будучи в прошлый четверг в Софии, он назвал се...",0,NaN,NaN,NaN,NaN,NaN,NaN
1,Работа с двухбайтовыми наборами символов — про...,1,92:99,83:91,103:109,127:127,119:124,127:134
2,"Заместитель Генерального секретаря подчеркнул,...",0,NaN,NaN,NaN,NaN,NaN,NaN
3,Продажа недвижимости из собственных портфелей ...,0,NaN,NaN,NaN,NaN,NaN,NaN
4,"Новым является то, что повышенное давление кон...",0,NaN,NaN,NaN,NaN,NaN,NaN


Считаем.

In [4]:
train.groupby(["class"]).count()

,text,cV,cR1,cR2,V,R1,R2
class,,,,,,,
0,10864,0,0,0,0,0,0
1,5542,5542,5541,5465,5542,5542,5465


In [5]:
test.groupby(["class"]).count()

,text,cV,cR1,cR2,V,R1,R2
class,,,,,,,
0,1365,0,0,0,0,0,0
1,680,680,680,663,680,680,663


In [6]:
dev.groupby(["class"]).count()

,text,cV,cR1,cR2,V,R1,R2
class,,,,,,,
0,2760,0,0,0,0,0,0
1,1382,1382,1382,1355,1382,1382,1355


В примерно двух третях текстов train (10864) гэппинг есть, в одной трети (5542) -- нет. 
Такие же пропорции (1356/680, 2760/1382) сохраняются для test и dev.

### 2. Анализ разметки (2 балла)
Стандартный пример содержит по паре оффсетов для каждой метки **cV, cR1, cR2, V, R1, R2** , но есть и другие варианты. Посчитайте сколько их и какие случаи им соответствуют.
Представьте результаты в понятном виде, например в таблице.

In [7]:
def count_offset(df, tag, tag_counter):
    df[tag_counter] = "no_offsets"
    for i, line in enumerate(df[tag]):
        if type(df[tag].iloc[i]) == str:
            df[tag_counter].iloc[i] = df[tag].iloc[i].count(":") 
    return df

In [8]:
tags = ["cV", "cR1", "cR2", "V", "R1", "R2"]
tag_counters = ["cV_amount", "cR1_amount", "cR2_amount", "V_amount", "R1_amount", "R2_amount"]

Считаем количество каждого из тегов для всех таблиц.

In [10]:
%%time
for i, tag in enumerate(tags):
    count_offset(test, tag, tag_counters[i])

CPU times: user 1min 13s, sys: 8.04 ms, total: 1min 13s
Wall time: 1min 13s


In [11]:
%%time
for i, tag in enumerate(tags):
    count_offset(dev, tag, tag_counters[i])

CPU times: user 2min 28s, sys: 8.03 ms, total: 2min 28s
Wall time: 2min 28s


In [12]:
%%time
for i, tag in enumerate(tags):
    count_offset(train, tag, tag_counters[i])

CPU times: user 11min 39s, sys: 256 ms, total: 11min 39s
Wall time: 11min 39s


In [13]:
train.head() 

,text,class,cV,cR1,cR2,V,R1,R2,cV_amount,cR1_amount,cR2_amount,V_amount,R1_amount,R2_amount
0,"Будучи в прошлый четверг в Софии, он назвал се...",0,NaN,NaN,NaN,NaN,NaN,NaN,no_offsets,no_offsets,no_offsets,no_offsets,no_offsets,no_offsets
1,Работа с двухбайтовыми наборами символов — про...,1,92:99,83:91,103:109,127:127,119:124,127:134,1,1,1,1,1,1
2,"Заместитель Генерального секретаря подчеркнул,...",0,NaN,NaN,NaN,NaN,NaN,NaN,no_offsets,no_offsets,no_offsets,no_offsets,no_offsets,no_offsets
3,Продажа недвижимости из собственных портфелей ...,0,NaN,NaN,NaN,NaN,NaN,NaN,no_offsets,no_offsets,no_offsets,no_offsets,no_offsets,no_offsets
4,"Новым является то, что повышенное давление кон...",0,NaN,NaN,NaN,NaN,NaN,NaN,no_offsets,no_offsets,no_offsets,no_offsets,no_offsets,no_offsets


Посмотрим на то, сколько каких тегов встречается. 

In [14]:
train.groupby(["cV_amount"]).count()

,text,class,cV,cR1,cR2,V,R1,R2,cR1_amount,cR2_amount,V_amount,R1_amount,R2_amount
cV_amount,,,,,,,,,,,,,
1,5542,5542,5542,5541,5465,5542,5542,5465,5542,5542,5542,5542,5542
no_offsets,10864,10864,0,0,0,0,0,0,10864,10864,10864,10864,10864


Вершина (если гэппинг есть) всегда одна. 

In [15]:
train.groupby(["V_amount"]).count()

,text,class,cV,cR1,cR2,V,R1,R2,cV_amount,cR1_amount,cR2_amount,R1_amount,R2_amount
V_amount,,,,,,,,,,,,,
1,5173,5173,5173,5172,5096,5173,5173,5096,5173,5173,5173,5173,5173
2,315,315,315,315,315,315,315,315,315,315,315,315,315
3,45,45,45,45,45,45,45,45,45,45,45,45,45
4,7,7,7,7,7,7,7,7,7,7,7,7,7
5,2,2,2,2,2,2,2,2,2,2,2,2,2
no_offsets,10864,10864,0,0,0,0,0,0,10864,10864,10864,10864,10864


Количество эллиптированных предикатов меняется от 0 (10864 случая при отсутствии гэппинга) до 5 (2 случая).
Посмотрим на редкие случаи.

In [16]:
filter_5 = train['V_amount'] == 5
for sentence in train.loc[filter_5]["text"]:
    print (sentence)

Домохозяйки становятся телеведущими, телеведущие - писателями, писатели - музыкантами, музыканты - актерами, актеры - политиками, а гастарбайтеры - мастерами суши.
Индекс Xetra DAX-30 (Германия) снизился на 6%, индекс CAC-40 (Франция) — на 5,6%, индекс FTSE-100 (Великобритания) — на 3%, индекс Nikkei 225 (Япония) — на 8,2%, индекс Hang Seng (Гонконг) — на 4,5%, индекс Kospi (Республика Корея) — на 10,1%.


In [17]:
filter_4 = train['V_amount'] == 4
for sentence in train.loc[filter_4]["text"]:
    print (sentence)

Индекс Xetra DAX-30 (Германия) повысился на 4,9%, индекс CAC-40 (Франция) — на 3,7%, индекс FTSE-100 (Великобритания) — на 1,6%, индекс Nikkei 225 (Япония) — на 8,9%, индекс Hang Seng (Гонконг) — на 2,1%.
В течение предпоследней недели июня в общем объеме заключенных сделок на долю акций РАО «ЕЭС России» пришлось около 33,2%, на долю акций «Мосэнерго» –16,2%, на долю акций «Сургутнефтегаз» – 12,3%, на долю акций «Лукойл» – 11,6%, на долю акций «Сибнефть» – 6,0%.
В явлениях погоды ветер соответствует дереву, тепло - огню, влажность - земле, сухость - металлу и холод - воде.
Из общего портфеля заказов РОЭ 42,5% приходится на технику ВВС, 23% - на военно-морскую технику, 22% - на системы ПВО, 9% - на технику для сухопутных войск и 3,5% - на прочую продукцию.
Земля (почва) соответствует селезенке, металл - легким, вода - почкам, дерево - печени и огонь - сердцу.
Обычно звук чэн произносится пациентами с поражениями сердца, звук шэн — пациентами с болезнями легких, звук гуо — пациентами с б

Много акций и процентов, но сложно говорить о каких-то серьезных закономерностях. Вне тегов в предложении практически ничего нет. 

Посмотрим на несколько предложений с одним эллиптированным глаголом. 

In [18]:
filter_1 = train['V_amount'] == 1
for sentence in train.loc[filter_1]["text"][0:5]:
    print (sentence)

Работа с двухбайтовыми наборами символов — просто кошмар для программиста, так как часть их состоит из одного байта, а часть — из двух.
Выворачивает наизнанку, запад превращает в юг, а север — в восток, меняет местами добро и зло, велит открыть своё сердце, ничего не требуя взамен.
Текущие обязательства компании в 2009 г. сократились на 16% - до 135.3 млн. грн., долгосрочные обязательства уменьшились на 1.7% - до 71.8 млн. грн., а дебиторская задолженность - на 3% до 87.7 млн. грн.
Первая конвенция была ратифицирована 80 процентами государств – членов МОТ, а вторая – 87 процентами этих государств.
Но Христианин сказал Упрямому: — Нет, сосед, лучше ты последуй примеру Сговорчивого. Мы в самом деле там получим то, о чем я сейчас говорил, а сверх того — великую славу.


Можно заметить, что здесь гораздо больше частей, не относящихся к интересующим нас тегам. Сами предложения довольно разнообразны. 

In [19]:
train.groupby(["cR1_amount"]).count()

,text,class,cV,cR1,cR2,V,R1,R2,cV_amount,cR2_amount,V_amount,R1_amount,R2_amount
cR1_amount,,,,,,,,,,,,,
1,5541,5541,5541,5541,5464,5541,5541,5464,5541,5541,5541,5541,5541
no_offsets,10865,10865,1,0,1,1,1,1,10865,10865,10865,10865,10865


In [20]:
train.groupby(["R1_amount"]).count()

,text,class,cV,cR1,cR2,V,R1,R2,cV_amount,cR1_amount,cR2_amount,V_amount,R2_amount
R1_amount,,,,,,,,,,,,,
1,5173,5173,5173,5172,5096,5173,5173,5096,5173,5173,5173,5173,5173
2,315,315,315,315,315,315,315,315,315,315,315,315,315
3,45,45,45,45,45,45,45,45,45,45,45,45,45
4,7,7,7,7,7,7,7,7,7,7,7,7,7
5,2,2,2,2,2,2,2,2,2,2,2,2,2
no_offsets,10864,10864,0,0,0,0,0,0,10864,10864,10864,10864,10864


Распределение для cR1 такое же, как и для cV1.

In [21]:
train.groupby(["cR2_amount"]).count()

,text,class,cV,cR1,cR2,V,R1,R2,cV_amount,cR1_amount,V_amount,R1_amount,R2_amount
cR2_amount,,,,,,,,,,,,,
1,5465,5465,5465,5464,5465,5465,5465,5465,5465,5465,5465,5465,5465
no_offsets,10941,10941,77,77,0,77,77,0,10941,10941,10941,10941,10941


In [22]:
train.groupby(["R2_amount"]).count()

,text,class,cV,cR1,cR2,V,R1,R2,cV_amount,cR1_amount,cR2_amount,V_amount,R1_amount
R2_amount,,,,,,,,,,,,,
1,5096,5096,5096,5095,5096,5096,5096,5096,5096,5096,5096,5096,5096
2,315,315,315,315,315,315,315,315,315,315,315,315,315
3,45,45,45,45,45,45,45,45,45,45,45,45,45
4,7,7,7,7,7,7,7,7,7,7,7,7,7
5,2,2,2,2,2,2,2,2,2,2,2,2,2
no_offsets,10941,10941,77,77,0,77,77,0,10941,10941,10941,10941,10941


Можно заметить, что в ряде предложений есть cR1, но нет cR2. Посмотрим на эти предложения. 

In [23]:
for sent in train[(train["cR2_amount"] == "no_offsets") & (train["cR1_amount"] != "no_offsets")]["text"][0:10]:
    print (sent)

Но Христианин сказал Упрямому: — Нет, сосед, лучше ты последуй примеру Сговорчивого. Мы в самом деле там получим то, о чем я сейчас говорил, а сверх того — великую славу.
Неправильное или слишком быстрое разогревание воска может привести к ожогам, а при хрупкости капилляров- к появлению сосудистых звездочек на коже.
И мы прогнозируем увеличение товарооборота на 70 процентов, а по информации российской стороны – на 40 процентов.
Списание дебиторской задолженности по финансовому лизингу может произойти только после получения одобрения со стороны Совета Директоров, а в некоторых случаях - при наличии соответствующего судебного решения.
Судья посмотрела на свои часы, затем – на меня.
В лучшем случае этот обычай представляется не более чем стремлением сохранить власть в руках семьи, а в худшем — источником генетического вырождения.
Круг в основном имеет голубоватый цвет и лишь по внешнему краю - красноватый.
Кто любит лето, кто весну,
Переступив через порог, я обнаружил небольшую площадку, 

Стало чуть яснее, что cR1 и cR2 -- это не левый и правый, а первый и второй. Также эти случаи кажутся менее очевидными, чем предыдущие (например, предложение "Судья посмотрела на свои часы, затем -- на меня" может показаться просто предложением с однородными дополнениями). Впрочем, таких случаев не очень много. Перейдем к разметке. 

### 3. Конвертер разметки-1 (1 балл)
Напишите функцию, переводящую разметку по оффсетам в IOB/IO/BIOS(на выбор) формат и обратно. Она пригодится вам при формировании входа для сети для решения классификации последовательности.

Напоминание про IOB(ES) разметку:
Каждому элементу последовательности (например токену или символу) сопоставляется одна из типов меток: O - outside, B - begin,
I - inside, E - end, s - single, в зависимости от того, входит ли этот элемент в интересующую нас подстроку. Например, в задаче NER метка B-LOC означает, что данный элемент - первый в подстроке, соответствующей названию локации. 

Пример подходящей для нашей задачи IOB-разметки (на уровне токенов):

| Тогда | я   | принял  | ее    | за    | итальянку  | а   | его  | за   | шведа |
| :---  | :---| :---    | :---  | :---  | :---       | :---| :--- | :--- | :---  |
| O     | O   | B-cV    | B-cR1 | B-cR2 | I-cR2      | O   | B-R1 | B-R2 | I-R2  |

Токенизация текста.

In [24]:
from nltk.tokenize import word_tokenize
from string import punctuation
from nltk.corpus import stopwords
punct = punctuation+'«»—…“”*№–'

def tokenize(text):
    
    words = word_tokenize(text)
    words = [word for word in words if word and word not in punct]
    words = [word.strip(punct) for word in words]
    return words

train['text_tokens'] = train['text'].apply(tokenize)

In [25]:
train.head()

,text,class,cV,cR1,cR2,V,R1,R2,cV_amount,cR1_amount,cR2_amount,V_amount,R1_amount,R2_amount,text_tokens
0,"Будучи в прошлый четверг в Софии, он назвал се...",0,NaN,NaN,NaN,NaN,NaN,NaN,no_offsets,no_offsets,no_offsets,no_offsets,no_offsets,no_offsets,"[Будучи, в, прошлый, четверг, в, Софии, он, на..."
1,Работа с двухбайтовыми наборами символов — про...,1,92:99,83:91,103:109,127:127,119:124,127:134,1,1,1,1,1,1,"[Работа, с, двухбайтовыми, наборами, символов,..."
2,"Заместитель Генерального секретаря подчеркнул,...",0,NaN,NaN,NaN,NaN,NaN,NaN,no_offsets,no_offsets,no_offsets,no_offsets,no_offsets,no_offsets,"[Заместитель, Генерального, секретаря, подчерк..."
3,Продажа недвижимости из собственных портфелей ...,0,NaN,NaN,NaN,NaN,NaN,NaN,no_offsets,no_offsets,no_offsets,no_offsets,no_offsets,no_offsets,"[Продажа, недвижимости, из, собственных, портф..."
4,"Новым является то, что повышенное давление кон...",0,NaN,NaN,NaN,NaN,NaN,NaN,no_offsets,no_offsets,no_offsets,no_offsets,no_offsets,no_offsets,"[Новым, является, то, что, повышенное, давлени..."


In [26]:
def splt(i):
    
    mask = train['class'] == i
    df1, df0 = train[mask], train[~mask]
    
    return df1

In [27]:
train_ellipsis = splt(1)

In [28]:
train_ellipsis.head()

,text,class,cV,cR1,cR2,V,R1,R2,cV_amount,cR1_amount,cR2_amount,V_amount,R1_amount,R2_amount,text_tokens
1,Работа с двухбайтовыми наборами символов — про...,1,92:99,83:91,103:109,127:127,119:124,127:134,1,1,1,1,1,1,"[Работа, с, двухбайтовыми, наборами, символов,..."
5,"Выворачивает наизнанку, запад превращает в юг,...",1,30:40,24:29,41:45,57:57,49:54,57:65,1,1,1,1,1,1,"[Выворачивает, наизнанку, запад, превращает, в..."
7,Текущие обязательства компании в 2009 г. сокра...,1,109:120,82:108,121:148,180:180,152:177,180:202,1,1,1,1,1,1,"[Текущие, обязательства, компании, в, 2009, г,..."
8,Первая конвенция была ратифицирована 80 процен...,1,22:36,0:16,37:74,87:87,78:84,87:116,1,1,1,1,1,1,"[Первая, конвенция, была, ратифицирована, 80, ..."
14,"Но Христианин сказал Упрямому: — Нет, сосед, л...",1,105:112,113:139,NaN,156:156,156:169,NaN,1,1,no_offsets,1,1,no_offsets,"[Но, Христианин, сказал, Упрямому, Нет, сосед,..."


In [29]:
train_ellipsis['index'] = train_ellipsis.index

/home/laidhimonthegreen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [30]:
from operator import itemgetter

def get_intervals(df, n):
    tags = ['cV', 'cR1', 'cR2', 'R1', 'R2']
    dict_tags = [tag for tag in tags if type(df[tag][n]) == str]
    
    intervals = []
    for tag in dict_tags:
        el = df[tag][n].split(' ')
        for l in el:
            inter = l.split(':')
            inter = [int(i) for i in inter]
            
            intervals.append((tag, inter))
          
    intervals = sorted(intervals, key=itemgetter(1), reverse = True)
    
    return intervals

In [31]:
get_intervals(train_ellipsis, train_ellipsis['index'].iloc[42])

[('R2', [147, 153]),
 ('R1', [118, 144]),
 ('R2', [109, 115]),
 ('R1', [77, 106]),
 ('cR2', [69, 75]),
 ('cV', [60, 68]),
 ('cR1', [7, 33])]

Конвертируем токены в теги. 

In [32]:
from collections import Counter

def get_elem(df, num, el):
    if type(df[el][num]) == str:
        ls = df[el][num].split(' ')
        words = []
        for l in ls: 
            el = df['text'][num][int(l.split(':')[0]):int(l.split(':')[1])]
            words.append(el)
    else:
        words = '-'
    return words


def replace(df, i):

    tags = ['cV', 'cR1', 'cR2', 'R1', 'R2']
    
    s = df['text'][i]
    
    tags_dict = get_intervals(df, i)
    
    for n, t in enumerate(tags_dict):
        s1 = s[:(t[1][0])]+t[0]+s[t[1][1]:]
        s = s1
        
    return s


def convert(df, i):

    tags = ['cV', 'cR1', 'cR2', 'R1', 'R2']
    
    s = replace(df, i)
    
    tokens = tokenize(s)
    #print(tokens)
    
    tok = []
    
    for token in tokens:
        if token not in tags:
            tok.append('O')
        elif token in tags:
            tok.append(token)
            
    #print(tok)
            
           
    lst = []
    
    lst_2 = []
    
    for n, t in enumerate(tok):
        
        for tag in tags:
            
            if t == 'O':
                lst.append(t)
                break
            
            elif t == tag:
                if tag not in lst_2:
                    el = get_elem(df, i, tag)[0]
                    count = len(tokenize(el))
                    if tokenize(el) != [] and count == 1:
                        lst.append('B-'+tag)
                        lst_2.append(tag)
                        
                    elif tokenize(el) != [] and count > 1:
                        lst.append('B-'+tag)
                        lst_2.append(tag)
                        for m in range(count - 1):
                            lst.append('I-'+tag)
        
                elif tag in lst_2:
                    el = get_elem(df, i, tag)[lst_2.count(tag)]
                    count = len(tokenize(el))
                    if tokenize(el) != [] and count == 1:
                        lst.append('B-'+tag)
                        lst_2.append(tag)
                        
                    elif tokenize(el) != [] and count > 1:
                        lst.append('B-'+tag)
                        lst_2.append(tag)
                        for m in range(count - 1):
                            lst.append('I-'+tag)
                    
    return lst

In [33]:
convert(train_ellipsis, train_ellipsis['index'].iloc[42])

['O',
 'B-cR1',
 'I-cR1',
 'O',
 'O',
 'O',
 'O',
 'B-cV',
 'B-cR2',
 'B-R1',
 'I-R1',
 'I-R1',
 'I-R1',
 'B-R2',
 'O',
 'B-R1',
 'I-R1',
 'I-R1',
 'B-R2']

Посчитаем количество слов и тегов, чтобы проверить, как работает наш конвертер.

In [34]:
tags = []
for i in range(len(train_ellipsis) - 1):
    t = convert(train_ellipsis, train_ellipsis['index'].iloc[i])
    tags.extend(t)
    
words = []
for i in range(len(train_ellipsis) - 1):
    w = train_ellipsis['text_tokens'].iloc[i]
    words.extend(w)

In [35]:
len(words), len(tags)

(101638, 101361)

Где-то, похоже, есть ошибки. Проверим. 

In [38]:
def check(df):
    false_convert = []
    for i in range(len(df) - 1):
        t = convert(df, df['index'].iloc[i])
        w = df['text_tokens'].iloc[i]
        if len(t) != len(w):
            false_convert.append(i)
    return false_convert

In [39]:
check(train_ellipsis)

[162,
 172,
 238,
 346,
 516,
 539,
 556,
 665,
 676,
 729,
 735,
 1004,
 1049,
 1076,
 1191,
 1283,
 1343,
 1480,
 1578,
 1710,
 1790,
 1808,
 1945,
 1956,
 1960,
 2043,
 2071,
 2257,
 2275,
 2353,
 2423,
 2519,
 2750,
 2856,
 2903,
 2911,
 2921,
 3029,
 3351,
 3396,
 3401,
 3447,
 3605,
 4187,
 4251,
 4310,
 4401,
 4489,
 4528,
 4566,
 4814,
 4889,
 4897,
 5073,
 5173,
 5185,
 5227,
 5240,
 5269,
 5397]

In [40]:
train_ellipsis['text_tokens'].iloc[172]

['Трусливые',
 'и',
 'жадные',
 'изберут',
 'первую',
 'мужественные',
 'и',
 'щедрые-вторую',
 '']

In [41]:
convert(train_ellipsis, train_ellipsis['index'].iloc[172])

['B-cR1', 'I-cR1', 'I-cR1', 'B-cV', 'B-cR2', 'O', 'O']

Проблема в токенизации. Тем не менее, для подавляющего большинства предложений конвертер работет правильно. Для решения нашей задачи мы можем просто удалить предложения с пунктуационными проблемами из корпуса.

### 4. Конвертер разметки-2 (1 балл)
Напишите функцию, переводящую разметку по оффсетам в скобочную запись. Она пригодится вам при анализе получаемых результатов.

In [44]:
#база данных для тренировки конвертера
train_ellipsis.head()

,text,class,cV,cR1,cR2,V,R1,R2,cV_amount,cR1_amount,cR2_amount,V_amount,R1_amount,R2_amount,text_tokens,index
1,Работа с двухбайтовыми наборами символов — про...,1,92:99,83:91,103:109,127:127,119:124,127:134,1,1,1,1,1,1,"[Работа, с, двухбайтовыми, наборами, символов,...",1
5,"Выворачивает наизнанку, запад превращает в юг,...",1,30:40,24:29,41:45,57:57,49:54,57:65,1,1,1,1,1,1,"[Выворачивает, наизнанку, запад, превращает, в...",5
7,Текущие обязательства компании в 2009 г. сокра...,1,109:120,82:108,121:148,180:180,152:177,180:202,1,1,1,1,1,1,"[Текущие, обязательства, компании, в, 2009, г,...",7
8,Первая конвенция была ратифицирована 80 процен...,1,22:36,0:16,37:74,87:87,78:84,87:116,1,1,1,1,1,1,"[Первая, конвенция, была, ратифицирована, 80, ...",8
14,"Но Христианин сказал Упрямому: — Нет, сосед, л...",1,105:112,113:139,NaN,156:156,156:169,NaN,1,1,no_offsets,1,1,no_offsets,"[Но, Христианин, сказал, Упрямому, Нет, сосед,...",14


In [59]:
train_ellipsis['text'].iloc[42]

'Индекс промышленного производства за январь-февраль 2008 г. составил 106,0%, инвестиций в основной капитал - 120,2% и оборота розничной торговли - 116,3%.'

In [45]:
get_intervals(train_ellipsis, train_ellipsis['index'].iloc[42])

[('R2', [147, 153]),
 ('R1', [118, 144]),
 ('R2', [109, 115]),
 ('R1', [77, 106]),
 ('cR2', [69, 75]),
 ('cV', [60, 68]),
 ('cR1', [7, 33])]

In [52]:
#функция, которая меняет формат оффсета для удобной работы со срезами
def offset_to_slices(line):
    line = [[int(x) for x in x.split(":")] for x in line.split(" ")]
    return line
    
offset_to_slices("44:46 50:57 60:62") 

[[44, 46], [50, 57], [60, 62]]

In [67]:
train_ellipsis['text'].iloc[42][109:115]

'120,2%'

In [179]:
offset_to_slices(train_ellipsis['R2'].iloc[42]) 

[[109, 115], [147, 153]]

In [79]:
offset_to_slices(train_ellipsis["cR1"].iloc[1])

[[24, 29]]

In [289]:
#функция, которая возвращает участок строки с тегом
def tagging(sentence, interval):  
    if not sentence[interval[1][0]:interval[1][1]]: 
        part_of_line = interval[0] + "["+sentence[interval[1][0]:interval[1][1]]+"] "
    else:
        part_of_line = interval[0]+"[ "+sentence[interval[1][0]:interval[1][1]]+" "+interval[0]+"]" 
    return part_of_line

In [292]:
tagging(train_ellipsis["text"].iloc[42], ('R2', [147, 153]))

'R2[ 116,3% R2]'

In [293]:
def tagging_for_dataline(df, n, interval):
    return tagging(df["text"].iloc[n], interval)

In [294]:
tagging_for_dataline(train_ellipsis, 42, ('R2', [147, 153]))

'R2[ 116,3% R2]'

In [295]:
#функция, которая разбивает предложение
def slice_from_sentence(sentence, slce):
    part_of_line = "" 
    part_of_line = sentence[slce[0]:slce[1]]
    left_line = sentence[:slce[0]]
    right_line = sentence[slce[1]:]
    return (left_line, part_of_line, right_line)

In [296]:
def slice_for_dataline(df, n, slce):
    return slice_from_sentence(df["text"].iloc[n], slce)

In [297]:
slice_for_dataline(train_ellipsis, 42, offset_to_slices(train_ellipsis["R2"].iloc[42])[0])

('Индекс промышленного производства за январь-февраль 2008 г. составил 106,0%, инвестиций в основной капитал - ',
 '120,2%',
 ' и оборота розничной торговли - 116,3%.')

Оформляем окончательный вариант конвертера. 

In [301]:
def getting_brackets(sentence, df, n): 
    intervals = get_intervals(df, df['index'].iloc[n])  
    for part in intervals:   
        left = slice_from_sentence(sentence, part[1])[0]  
        centre = slice_from_sentence(sentence, part[1])[1]
        right = slice_from_sentence(sentence, part[1])[2]  
        sentence = left+tagging(sentence, part) + right   
    return sentence

In [302]:
getting_brackets(train_ellipsis["text"].iloc[42], train_ellipsis, 42)

'Индекс cR1[ промышленного производства cR1] за январь-февраль 2008 г. cV[ составил cV] cR2[ 106,0% cR2], R1[ инвестиций в основной капитал R1] - R2[ 120,2% R2] и R1[ оборота розничной торговли R1] - R2[ 116,3% R2].'

### 5. Бинарная классификация (6 баллов (+2))
Постройте простую полносвязную сеть, берущую на вход все предложение и на выходе предсказывающую наличие или отсутствие гэппинга в предложении. Признаки: использование эмбеддингов +1 балл, использование pos-тэгов +1 балл

In [303]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
from nltk import word_tokenize

In [304]:
train.head()

,text,class,cV,cR1,cR2,V,R1,R2,cV_amount,cR1_amount,cR2_amount,V_amount,R1_amount,R2_amount,text_tokens
0,"Будучи в прошлый четверг в Софии, он назвал се...",0,NaN,NaN,NaN,NaN,NaN,NaN,no_offsets,no_offsets,no_offsets,no_offsets,no_offsets,no_offsets,"[Будучи, в, прошлый, четверг, в, Софии, он, на..."
1,Работа с двухбайтовыми наборами символов — про...,1,92:99,83:91,103:109,127:127,119:124,127:134,1,1,1,1,1,1,"[Работа, с, двухбайтовыми, наборами, символов,..."
2,"Заместитель Генерального секретаря подчеркнул,...",0,NaN,NaN,NaN,NaN,NaN,NaN,no_offsets,no_offsets,no_offsets,no_offsets,no_offsets,no_offsets,"[Заместитель, Генерального, секретаря, подчерк..."
3,Продажа недвижимости из собственных портфелей ...,0,NaN,NaN,NaN,NaN,NaN,NaN,no_offsets,no_offsets,no_offsets,no_offsets,no_offsets,no_offsets,"[Продажа, недвижимости, из, собственных, портф..."
4,"Новым является то, что повышенное давление кон...",0,NaN,NaN,NaN,NaN,NaN,NaN,no_offsets,no_offsets,no_offsets,no_offsets,no_offsets,no_offsets,"[Новым, является, то, что, повышенное, давлени..."


#### POS-вектора для предложений.

Будем учитывать pos-теги и пунктуацию. 

In [305]:
def pos_tagging (text):
    
    words = word_tokenize(text)
    new_words = []
    for word in words:
        if morph.parse(word)[0].tag.POS != None:
            new_words.append(str(morph.parse(word)[0].tag.POS))        
        elif word in punct:
            new_words.append(word)
        else:
            continue

    return new_words

In [306]:
pos_tagging (train["text"][1])

['NOUN',
 'PREP',
 'ADJF',
 'NOUN',
 'NOUN',
 '—',
 'PRCL',
 'NOUN',
 'PREP',
 'NOUN',
 ',',
 'CONJ',
 'CONJ',
 'NOUN',
 'NPRO',
 'VERB',
 'PREP',
 'ADJF',
 'NOUN',
 ',',
 'CONJ',
 'NOUN',
 '—',
 'PREP',
 'NUMR',
 '.']

In [307]:
train["pos_text"] = train["text"].apply(pos_tagging)

In [308]:
test["pos_text"] = test["text"].apply(pos_tagging)
dev["pos_text"] = dev["text"].apply(pos_tagging)

Смотрим на теги, которые у нас есть.

In [309]:
all_tags = ["~"]
for pos_tags in train["pos_text"]:
    all_tags += pos_tags   
for tags in test["pos_text"]:
    all_tags += pos_tags 

In [310]:
dict_all_tags = {}
unique_tags = list(set(all_tags))
for i, tag in enumerate(unique_tags):
    dict_all_tags[tag] = i   

In [311]:
amount_tags = len(set(all_tags))

In [312]:
amount_tags

54

In [313]:
snt = pos_tagging ("Котики почти так же хороши, как компьютерная лингвистика")

In [314]:
snt

['NOUN', 'ADVB', 'CONJ', 'PRCL', 'ADJS', ',', 'CONJ', 'ADJF', 'NOUN']

Хорошо, всё работает. 

Создаём для каждого предложения вектор формата "количество слов с данным тегом или данных знаков препинания в предложении".

In [315]:
import numpy as np
import collections
c = collections.Counter()

In [316]:
v = np.zeros((1, amount_tags)) 
v

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.]])

In [317]:
def vectorize(sent):    
    v = np.zeros((1, amount_tags)) 
    for wrd in sent:
        v[0][dict_all_tags[wrd]] = sent.count(wrd) 
    return v[0]

In [318]:
train["pos_vector"] = train["pos_text"].apply(vectorize)

In [319]:
test["pos_vector"] = test["pos_text"].apply(vectorize) 

In [320]:
train

,text,class,cV,cR1,cR2,V,R1,R2,cV_amount,cR1_amount,cR2_amount,V_amount,R1_amount,R2_amount,text_tokens,pos_text,pos_vector
0,"Будучи в прошлый четверг в Софии, он назвал се...",0,NaN,NaN,NaN,NaN,NaN,NaN,no_offsets,no_offsets,no_offsets,no_offsets,no_offsets,no_offsets,"[Будучи, в, прошлый, четверг, в, Софии, он, на...","[GRND, PREP, ADJF, NOUN, PREP, NOUN, ,, NPRO, ...","[8.0, 0.0, 2.0, 0.0, 3.0, 0.0, 0.0, 4.0, 1.0, ..."
1,Работа с двухбайтовыми наборами символов — про...,1,92:99,83:91,103:109,127:127,119:124,127:134,1,1,1,1,1,1,"[Работа, с, двухбайтовыми, наборами, символов,...","[NOUN, PREP, ADJF, NOUN, NOUN, —, PRCL, NOUN, ...","[8.0, 0.0, 2.0, 0.0, 1.0, 0.0, 0.0, 4.0, 0.0, ..."
2,"Заместитель Генерального секретаря подчеркнул,...",0,NaN,NaN,NaN,NaN,NaN,NaN,no_offsets,no_offsets,no_offsets,no_offsets,no_offsets,no_offsets,"[Заместитель, Генерального, секретаря, подчерк...","[NOUN, ADJF, GRND, VERB, ,, CONJ, NOUN, NOUN, ...","[14.0, 0.0, 4.0, 0.0, 1.0, 1.0, 0.0, 2.0, 0.0,..."
3,Продажа недвижимости из собственных портфелей ...,0,NaN,NaN,NaN,NaN,NaN,NaN,no_offsets,no_offsets,no_offsets,no_offsets,no_offsets,no_offsets,"[Продажа, недвижимости, из, собственных, портф...","[NOUN, NOUN, PREP, ADJF, NOUN, VERB, INFN, NPR...","[9.0, 0.0, 4.0, 0.0, 1.0, 0.0, 0.0, 5.0, 0.0, ..."
4,"Новым является то, что повышенное давление кон...",0,NaN,NaN,NaN,NaN,NaN,NaN,no_offsets,no_offsets,no_offsets,no_offsets,no_offsets,no_offsets,"[Новым, является, то, что, повышенное, давлени...","[ADJF, VERB, CONJ, ,, CONJ, ADJF, NOUN, PRTF, ...","[38.0, 0.0, 12.0, 0.0, 2.0, 3.0, 0.0, 16.0, 0...."
5,"Выворачивает наизнанку, запад превращает в юг,...",1,30:40,24:29,41:45,57:57,49:54,57:65,1,1,1,1,1,1,"[Выворачивает, наизнанку, запад, превращает, в...","[VERB, ADVB, ,, NOUN, VERB, PREP, NOUN, ,, CON...","[6.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 2.0, 0.0, ..."
6,Специалисты отметили «грамотный подход к ландш...,0,NaN,NaN,NaN,NaN,NaN,NaN,no_offsets,no_offsets,no_offsets,no_offsets,no_offsets,no_offsets,"[Специалисты, отметили, грамотный, подход, к, ...","[NOUN, VERB, «, ADJF, NOUN, PREP, NOUN, », ,, ...","[7.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, ..."
7,Текущие обязательства компании в 2009 г. сокра...,1,109:120,82:108,121:148,180:180,152:177,180:202,1,1,1,1,1,1,"[Текущие, обязательства, компании, в, 2009, г,...","[ADJF, NOUN, NOUN, PREP, VERB, PREP, %, -, PRE...","[7.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 7.0, 0.0, ..."
8,Первая конвенция была ратифицирована 80 процен...,1,22:36,0:16,37:74,87:87,78:84,87:116,1,1,1,1,1,1,"[Первая, конвенция, была, ратифицирована, 80, ...","[ADJF, NOUN, VERB, PRTS, NOUN, NOUN, –, NOUN, ...","[7.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
9,"Ей свиновой надо чаще напоминать, а то столько...",0,NaN,NaN,NaN,NaN,NaN,NaN,no_offsets,no_offsets,no_offsets,no_offsets,no_offsets,no_offsets,"[Ей, свиновой, надо, чаще, напоминать, а, то, ...","[NPRO, ADJF, PRED, NOUN, INFN, ,, CONJ, CONJ, ...","[4.0, 0.0, 2.0, 0.0, 1.0, 0.0, 0.0, 2.0, 0.0, ..."


#### пробуем представить предложение в виде вектора с помощью эмбеддингов

нормализация текста*

*здесь очень много взято из тетрадки про эмбеддинги (см. источники внизу)

In [321]:
from ufal.udpipe import Model, Pipeline, ProcessingError

In [322]:
model_path = ("russian.udpipe")
model = Model.load(model_path)
pipeline = Pipeline(model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')

In [323]:
def preprocess_text(text, add_pos=True): 
    parsed_text = []
    for par in pipeline.process(text).split('\n\n')[:~0]:
        for word_parse in par.split('\n'):
            if word_parse[0].isdigit():
                word_data = word_parse.split('\t')
                lemma = word_data[2]
                pos = word_data[3]
                if str(pos) != 'PUNCT':
                    if add_pos:
                        parsed_text.append('{}_{}'.format(lemma.lower(), pos))
                    else:
                        parsed_text.append(lemma)
    return parsed_text

In [324]:
text = "Кот идет по улице"

In [325]:
preprocess_text(text)

['кот_NOUN', 'идти_VERB', 'по_ADP', 'улица_NOUN']

In [326]:
%%time
train["tagged_text"] = train["text"].apply(preprocess_text)

CPU times: user 3min 58s, sys: 80 ms, total: 3min 58s
Wall time: 3min 58s


In [327]:
dev["tagged_text"] = dev["text"].apply(preprocess_text)
test["tagged_text"] = test["text"].apply(preprocess_text)

перейдем к векторам

In [328]:
from gensim.models import KeyedVectors, Word2Vec

In [329]:
vectors = KeyedVectors.load_word2vec_format("180/model.txt")

In [330]:
def get_vector(sent, model):
    vector = np.zeros(shape=model.vector_size)
    counter = 0
    for word in sent:
        if word in model.vocab: 
            try:
                vector = np.add(vector, model[word])
                counter += 1
            except KeyError:
                pass
    if counter == 0:
        return vector.squeeze()
    return (vector / counter).squeeze()

In [331]:
train_vectors = [get_vector(text, vectors) for text in train.tagged_text] 

In [332]:
test_vectors = [get_vector(text, vectors) for text in test.tagged_text] 

In [333]:
dev_vectors = [get_vector(text, vectors) for text in dev.tagged_text] 

In [334]:
train["embedded_vectors"] = train_vectors
test["embedded_vectors"] = test_vectors
dev["embedded_vectors"] = dev_vectors

In [335]:
train

,text,class,cV,cR1,cR2,V,R1,R2,cV_amount,cR1_amount,cR2_amount,V_amount,R1_amount,R2_amount,text_tokens,pos_text,pos_vector,tagged_text,embedded_vectors
0,"Будучи в прошлый четверг в Софии, он назвал се...",0,NaN,NaN,NaN,NaN,NaN,NaN,no_offsets,no_offsets,no_offsets,no_offsets,no_offsets,no_offsets,"[Будучи, в, прошлый, четверг, в, Софии, он, на...","[GRND, PREP, ADJF, NOUN, PREP, NOUN, ,, NPRO, ...","[8.0, 0.0, 2.0, 0.0, 3.0, 0.0, 0.0, 4.0, 1.0, ...","[быть_AUX, в_ADP, прошлый_ADJ, четверг_NOUN, в...","[0.015995742245153946, -0.0807239284569567, 0...."
1,Работа с двухбайтовыми наборами символов — про...,1,92:99,83:91,103:109,127:127,119:124,127:134,1,1,1,1,1,1,"[Работа, с, двухбайтовыми, наборами, символов,...","[NOUN, PREP, ADJF, NOUN, NOUN, —, PRCL, NOUN, ...","[8.0, 0.0, 2.0, 0.0, 1.0, 0.0, 0.0, 4.0, 0.0, ...","[работа_NOUN, с_ADP, двухбайтовыми_ADJ, набора...","[-0.30684544808334774, -0.18797976730598343, 1..."
2,"Заместитель Генерального секретаря подчеркнул,...",0,NaN,NaN,NaN,NaN,NaN,NaN,no_offsets,no_offsets,no_offsets,no_offsets,no_offsets,no_offsets,"[Заместитель, Генерального, секретаря, подчерк...","[NOUN, ADJF, GRND, VERB, ,, CONJ, NOUN, NOUN, ...","[14.0, 0.0, 4.0, 0.0, 1.0, 1.0, 0.0, 2.0, 0.0,...","[заместитель_NOUN, генеральный_ADJ, секретарь_...","[-0.4773767518500487, -0.40940260564287506, 1...."
3,Продажа недвижимости из собственных портфелей ...,0,NaN,NaN,NaN,NaN,NaN,NaN,no_offsets,no_offsets,no_offsets,no_offsets,no_offsets,no_offsets,"[Продажа, недвижимости, из, собственных, портф...","[NOUN, NOUN, PREP, ADJF, NOUN, VERB, INFN, NPR...","[9.0, 0.0, 4.0, 0.0, 1.0, 0.0, 0.0, 5.0, 0.0, ...","[продажа_NOUN, недвижимости_NOUN, из_ADP, собс...","[0.201044961810112, 0.10514532178640365, -0.42..."
4,"Новым является то, что повышенное давление кон...",0,NaN,NaN,NaN,NaN,NaN,NaN,no_offsets,no_offsets,no_offsets,no_offsets,no_offsets,no_offsets,"[Новым, является, то, что, повышенное, давлени...","[ADJF, VERB, CONJ, ,, CONJ, ADJF, NOUN, PRTF, ...","[38.0, 0.0, 12.0, 0.0, 2.0, 3.0, 0.0, 16.0, 0....","[новый_PROPN, являться_AUX, то_PRON, что_SCONJ...","[0.8792008131933517, -0.158549407723909, 0.717..."
5,"Выворачивает наизнанку, запад превращает в юг,...",1,30:40,24:29,41:45,57:57,49:54,57:65,1,1,1,1,1,1,"[Выворачивает, наизнанку, запад, превращает, в...","[VERB, ADVB, ,, NOUN, VERB, PREP, NOUN, ,, CON...","[6.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 2.0, 0.0, ...","[выворачивает_VERB, наизнанку_NOUN, запад_NOUN...","[-0.09571943059563637, -0.1364715788513422, 0...."
6,Специалисты отметили «грамотный подход к ландш...,0,NaN,NaN,NaN,NaN,NaN,NaN,no_offsets,no_offsets,no_offsets,no_offsets,no_offsets,no_offsets,"[Специалисты, отметили, грамотный, подход, к, ...","[NOUN, VERB, «, ADJF, NOUN, PREP, NOUN, », ,, ...","[7.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, ...","[специалист_NOUN, отметить_VERB, «грамотный_AD...","[-0.8926077795525392, 0.48066549624005955, 1.6..."
7,Текущие обязательства компании в 2009 г. сокра...,1,109:120,82:108,121:148,180:180,152:177,180:202,1,1,1,1,1,1,"[Текущие, обязательства, компании, в, 2009, г,...","[ADJF, NOUN, NOUN, PREP, VERB, PREP, %, -, PRE...","[7.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 7.0, 0.0, ...","[текущий_NOUN, обязательство_NOUN, компания_NO...","[-1.2427225101273507, -0.8125229231081903, -0...."
8,Первая конвенция была ратифицирована 80 процен...,1,22:36,0:16,37:74,87:87,78:84,87:116,1,1,1,1,1,1,"[Первая, конвенция, была, ратифицирована, 80, ...","[ADJF, NOUN, VERB, PRTS, NOUN, NOUN, –, NOUN, ...","[7.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[первый_ADJ, конвенция_NOUN, быть_AUX, ратифиц...","[-0.5094917331423078, -0.07246802534375872, 0...."
9,"Ей свиновой надо чаще напоминать, а то столько...",0,NaN,NaN,NaN,NaN,NaN,NaN,no_offsets,no_offsets,no_offsets,no_offsets,no_offsets,no_offsets,"[Ей, свиновой, надо, чаще, напоминать, а, то, ...","[NPRO, ADJF, PRED, NOUN, INFN, ,, CONJ, CONJ, ...","[4.0, 0.0, 2.0, 0.0, 1.0, 0.0, 0.0, 2.0, 0.0, ...","[она_PRON, свиновой_ADJ, надо_VERB, часто_ADV,...

#### делаем саму сеть, используем perceptron из sklearn

In [337]:
from sklearn.metrics import f1_score, accuracy_score, precision_score

In [338]:
from sklearn.linear_model import Perceptron

In [339]:
new_clf = Perceptron(tol=1e-3, random_state=0)

In [340]:
new_clf.fit(train_vectors, train["class"])

Perceptron(alpha=0.0001, class_weight=None, early_stopping=False, eta0=1.0,
      fit_intercept=True, max_iter=None, n_iter=None, n_iter_no_change=5,
      n_jobs=None, penalty=None, random_state=0, shuffle=True, tol=0.001,
      validation_fraction=0.1, verbose=0, warm_start=False)

In [341]:
new_pred = new_clf.predict(test_vectors)

In [342]:
f1_score(new_pred, test["class"], average='micro')

0.6919315403422983

Не самый лучший результат.
Посмотрим для pos-тегов.

In [343]:
train_posvectors = list(train["pos_vector"])
test_posvectors = list(test["pos_vector"])

In [344]:
new_clf.fit(train_posvectors, train["class"])

Perceptron(alpha=0.0001, class_weight=None, early_stopping=False, eta0=1.0,
      fit_intercept=True, max_iter=None, n_iter=None, n_iter_no_change=5,
      n_jobs=None, penalty=None, random_state=0, shuffle=True, tol=0.001,
      validation_fraction=0.1, verbose=0, warm_start=False)

In [345]:
pos_pred = new_clf.predict(test_posvectors)

In [346]:
pos_pred

array([0, 1, 0, ..., 0, 0, 0])

In [347]:
f1_score(pos_pred, test["class"], average='micro')

0.769682151589242

Неожиданно лучше. Попробуем оценить результаты с помощью других метрик в следующем пункте. 

Спойлер: метрики показали, что мы ничего не смогли толком решить. Попробуем объединить векторы.

In [348]:
train_pos_emb = []
for i, vec in enumerate(train["pos_vector"]):
    train_pos_emb.append(list(np.hstack((vec, train_vectors[i]))))

In [349]:
train_pe = np.array(train_pos_emb)

In [350]:
test_pos_emb = []
for i, vec in enumerate(test["pos_vector"]):
    test_pos_emb.append(list(np.hstack((vec, test_vectors[i]))))
test_pe = np.array(test_pos_emb)

In [351]:
new_clf.fit(train_pe, train["class"])

Perceptron(alpha=0.0001, class_weight=None, early_stopping=False, eta0=1.0,
      fit_intercept=True, max_iter=None, n_iter=None, n_iter_no_change=5,
      n_jobs=None, penalty=None, random_state=0, shuffle=True, tol=0.001,
      validation_fraction=0.1, verbose=0, warm_start=False)

In [352]:
pe_pred = new_clf.predict(test_pe)

In [353]:
f1_score(pe_pred, test["class"], average='micro')

0.6757946210268949

Хуже, чем оба предыдущих варианта. 

### 6. Оценка результатов бинарной классификации (1 балл)
Обученной сеткой разметьте тестовые данные, с помощью готового скрипта agrr_metrics.py оцените полученный результат.
Пример вызова из командной строки:
> python3 agrr_metrics.py -b test.csv output.csv


In [354]:
predicted_file = test[["text", "class"]].copy()

In [355]:
predicted_file["class"] = pos_pred

In [356]:
correct_file = test[["text", "class"]].copy() 

In [357]:
predicted_file.to_csv("predicted_file", sep = "\t")
correct_file.to_csv("correct_file", sep = "\t")

##### Результат вызова функции:
Binary classification quality (f1-score): 0.5266331658291457  
Other metrics:    
Precision: 0.8317460317460318  
Recall: 0.38529411764705884  


С эмбеддингами.

In [358]:
emb_file = test[["text", "class"]].copy()
emb_file["class"] = new_pred
emb_file.to_csv("emb_file", sep = "\t")

Они еще более бесполезны. По-видимому, встроенные метрики склёрна льстили. 

Binary classification quality (f1-score): 0.4615384615384615  
Other metrics:  
Precision: 0.5510204081632653  
Recall: 0.39705882352941174  

Всё очень грустно. Вернемся к предыдущему пункту и посмотрим, что мы можем улучшить. 
Посмотрим на результат, учитывающий оба вектора. На метрике ф1 от склёрна он был одним из худших.

In [359]:
pe_file = test[["text", "class"]].copy()
pe_file["class"] = pe_pred
pe_file.to_csv("pe_file", sep = "\t")

Результат улучшился. Примем это за некрасивый бейзлайн.

Binary classification quality (f1-score): 0.6239364719228588  
Other metrics: 
Precision: 0.5078485687903971  
Recall: 0.8088235294117647  

### Пункты 7 и 8 отдельно в колабе.
[вот тут](https://colab.research.google.com/drive/1ioDsl1C9dph4Lv0GolZeFgjjSq2KiBgh)

### 9. Анализ результатов для бинарной классификации.
Проведите анализ ошибок. Посмотрите на fn, fp примеры, приведите несколько таких предложений в скобочной записи (если вы решали задачу классификации последовательности). Можно ли попытаться обобщить получаемые ошибки?

Посмотрим на false negatives (гэппинг не нашли там, где он есть) и false positives (гэппинг нашли там, где его нет).

In [362]:
pe_comparison = test[["text", "class"]].copy()

In [363]:
pe_comparison["predicted"] = pe_file["class"] 

In [365]:
pe_comparison.head()

,text,class,predicted
0,Изобретение относится к судостроению и касаетс...,0,0
1,"Эти состояния называют фазами воды, а превраще...",1,1
2,И должен ни единой долькой Не отступаться от ...,0,0
3,Он потребовал обеспечить полное осуществление ...,0,0
4,"По мнению местного пастора Элла Эбанкса, запре...",0,0


False negatives.

In [377]:
fns = []
for i, line in enumerate(pe_comparison["class"]):
        if pe_comparison["class"][i] == 1  and pe_comparison["predicted"].iloc[i] == 0:
            fns.append(pe_comparison["text"][i]) 

fns[:10]

['Описание структуры CREATESTRUCT приводится ниже, а описание структуры MDICREATESTRUCT — в разделе, посвященном сообщению WM_MDICREATE',
 'Друзья делают твой смех немного громче, твою улыбку немного ярче, и твою жизнь немного лучше',
 'Я сделаю тысячу шагов тебе на встречу, но не одного в догон.',
 'Я буду любить кого-нибудь так, как она меня,',
 'Но прежде чем она успела задать вопрос, а он ответить на него, в зал поспешно вошел пресвитер, худощавый мужчина с болезненным лицом.',
 'В казахском ауле сын не имеет право повышать голос на отца, отец не может повышать голос на деда, а дед на прадеда.',
 'Мы вернем Вашим шинам былую хватку, а Вам безопасность на дороге!',
 'Его глаза были испуганными и остекленевшими, дыхание неровным.',
 'На публичной странице можно написать контактные данные компании, в группе только ссылку на страницу руководителя.',
 'Авторами работы [68] показано, что при концентрации NaOH, равной 2 кмоль/м3, ДЭТАПК ЫО-4 кмоль/м3, экстрагента 1 • 10-2 кмоль/м3 и tK 10

Закономерностей в примерах найти не удалось (что неудивительно, ведь точность нашей сетки невысока). Некоторые из них (например, 'Я сделаю тысячу шагов тебе на встречу, но не одного в догон.') кажутся сомнительными примерами гэппинга, возможно, чаще неверно интерпретировались именно спорные предложения, но это только догадка. 

False positives.

In [379]:
fps = []
for i, line in enumerate(pe_comparison["class"]):
        if pe_comparison["class"][i] == 0  and pe_comparison["predicted"].iloc[i] == 1: 
            fps.append(pe_comparison["text"][i]) 

fps[:10]

['Международные правозащитники, некоторые национальные правительства и технократы из интернациональных организаций также играют в этом деле свою роль, стремясь получить больше бюрократических рычагов.',
 'Концы элементов закреплены в подвижных трубных решетках, установленных в стаканах.',
 'Знаете, обычно говорят, что девочка – это папина дочка.',
 'Подшипниковый щит 5 со стороны защитного кожуха 10 может быть выполнен с аксиальными сквозными каналами, снабженными обратными пневмоклапанами, открытыми в направлении электродвигателя к защитному кожуху.',
 'Продаю лес кругляк в Армавире и Краснодарском Крае - Кругляк, Пиловочник - - Главная',
 'Столь пристальное внимание спецслужб к нефти объясняется тем, что Кремль использует нефть в качестве инструмента восстановления своего влияния в мире, потерянного с распадом Советского Союза.',
 'Если мы хотим разложить функцию / в ряд в окрестности ее особой точки а, то для этой цели ряд Тейлора, очевидно, непригоден.',
 'Отходы, разделенные в вод

Опять сложно увидеть какие-либо закономерности.

### Источники заимствований кода:
    
1. Про эмбеддинги:
https://github.com/bakarov/seminars/blob/master/distributional-semantics/distributional-semantics.ipynb